In [22]:
import os
import json
import cv2
import glob
import numpy as np

In [23]:
IDS = [
    # 107884306,
    # 106036163,
    # 106244180,
    106551257,
    106390524,
    108312154,
    108054353,
    113748359,
    105988462,
    105811271,
    105722608,
    106432774,
    106964471,
    109933221,
    105908345,
    106340625,
    107046859,
    105722593,
    105880763,
    106403724,
    107017522
    ]

First let's download everything. This should just be run overnight.

In [24]:
def get_climb(id):
    with open('stats.js') as dataFile:
        data = dataFile.read()
        obj = data[data.find('{') : data.rfind('}')+1]
        jsonObj = json.loads(obj)
        return jsonObj[str(id)]

def download_vid(id):
    climb = get_climb(id)
    url = climb["src"]
    cmd = f"youtube-dl -o proc/{id}/orig {url}"
    os.system(cmd)

def make_dir_structure(id):
    os.makedirs(f"proc/{id}", exist_ok=True)
    os.makedirs(f"proc/{id}/clean", exist_ok=True)
    os.makedirs(f"proc/{id}/squares", exist_ok=True)
    os.makedirs(f"proc/{id}/mat_fw", exist_ok=True)
    os.makedirs(f"proc/{id}/mat_bw", exist_ok=True)
    os.makedirs(f"proc/{id}/mat_mean", exist_ok=True)
    os.makedirs(f"proc/{id}/hidden", exist_ok=True)

In [ ]:
for ID in IDS:
    make_dir_structure(ID)
    download_vid(ID)

Then we need to interact with the video and do some tracking. We'll track "center coordinates" with the mouse tracking over the belly button. We'll also size the square needed to contain the climber at their biggest.

In [26]:
def smooth(X, p=8):
    Xo = []
    for i,x in enumerate(X):
        if i < p:
            Xo.append(int(x))
        elif i > len(X) - p:
            Xo.append(int(x))
        else:
            Xo.append(int(np.mean(X[i-p:i+p+1])))
    return Xo

clicking = False
curX = 0
curY = 0
def run_interactive_tracking(ID):
    
    vid_file = glob.glob(f"proc/{ID}/orig*")[0]
    
    cmd = f"ffmpeg -i {vid_file} proc/{ID}/clean/%04d.png"
    os.system(cmd)

    im_files = sorted(glob.glob(f"proc/{ID}/clean/*.png"))

    posX = []
    posY = []
    L = 10

    global clicking, curX, curY
    curX = 0
    curY = 0
    clicking = False
    def onMouse(event, x, y, flags, param):
        global clicking, curX, curY
        if event == cv2.EVENT_LBUTTONDOWN:
            clicking = True
        elif event == cv2.EVENT_MOUSEMOVE:
            curX = x
            curY = y
        elif event == cv2.EVENT_LBUTTONUP:
            clicking = False

    i = 0
    while i < len(im_files):
        frame = cv2.imread(im_files[i])
        pframe = frame.copy()
        cv2.rectangle(pframe, (curX-L, curY-L), (curX+L, curY+L), (0,0,0), 5)
        cv2.line(pframe, (curX, curY+L), (curX, curY-L), (0,0,0), 2)
        cv2.line(pframe, (curX-L, curY), (curX+L, curY), (0,0,0), 2)

        cv2.imshow("Track That Thing", pframe)
        if i == 0:
            cv2.setMouseCallback("Track That Thing", onMouse)
        

        while True:
            pframe = frame.copy()
            cv2.rectangle(pframe, (curX-L, curY-L), (curX+L, curY+L), (0,0,0), 5)
            cv2.line(pframe, (curX, curY+L), (curX, curY-L), (0,0,0), 2)
            cv2.line(pframe, (curX-L, curY), (curX+L, curY), (0,0,0), 2)

            cv2.imshow("Track That Thing", pframe)
            kp = cv2.waitKey(100)
            
            if kp == ord('b'):
                L += 1
            elif kp == 81:
                posX.pop()
                posY.pop()
                i -= 1
                break
            elif clicking:
                posX.append(curX)
                posY.append(curY)
                i += 1
                break
    cv2.destroyAllWindows()
    
    posX = smooth(posX)
    posY = smooth(posY)

    with open(f'proc/{ID}/center_points.txt', 'w') as f:
        for (x,y) in zip(posX,posY):
            f.write(f"{int(x)},{int(y)}\n")
    
    with open(f'proc/{ID}/len.txt', 'w') as f:
            f.write(f"{int(L)}")

In [27]:
for ID in IDS[1:2]:
    run_interactive_tracking(ID)

ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

Now that we have the object tracked, we can do all the processing?

In [31]:
def get_crop_coords(xc,yc,L,w,h):
    x1 = xc - L
    x2 = xc + L
    y1 = yc - L
    y2 = yc + L

    if x1 < 0:
        x1 = 0
        x2 = x1 + 2*L
    elif x2 > w:
        x2 = w
        x1 = x2 - 2*L
    
    if y1 < 0:
        y1 = 0
        y2 = y1 + 2*L
    elif y2 > h:
        y2 = h
        y1 = y2 - 2*L
    
    return x1,x2,y1,y2

def crop_squares(id):
    im_files = sorted(glob.glob(f"proc/{id}/clean/*.png"))
    arr_cp = np.loadtxt(f"proc/{id}/center_points.txt", delimiter=",", dtype=int)
    L = np.loadtxt(f"proc/{id}/len.txt", delimiter=",", dtype=int)
    X = arr_cp[:,0]
    Y = arr_cp[:,1]

    for i in range(len(X)):
        im = cv2.imread(im_files[i])
        h,w = im.shape[0:2]

        x1,x2,y1,y2 = get_crop_coords(X[i], Y[i], L, w, h)


        out = im[y1:y2, x1:x2, :]
        cv2.imwrite(f"proc/{id}/squares/{i:04}.png", out)

def mat_squares(id):
    cmd = f"ffmpeg -y -i proc/{id}/squares/%4d.png -pix_fmt yuv420p proc/{id}/sq_fw.mp4"
    os.system(cmd)

    cmd = f"ffmpeg -y -i proc/{id}/squares/%4d.png -pix_fmt yuv420p -vf reverse proc/{id}/sq_bw.mp4"
    os.system(cmd)

    cmd = f"""
                python working/RobustVideoMatting/inference.py \\
                --device cuda \\
                --variant resnet50 \\
                --checkpoint working/RobustVideoMatting/rvm_resnet50.pth \\
                --input-source proc/{id}/sq_fw.mp4 \\
                --output-type png_sequence \\
                --output-alpha proc/{id}/mat_fw \\
                --seq-chunk 12
            """
    os.system(cmd)

    cmd = f"""
                python working/RobustVideoMatting/inference.py \\
                --device cuda \\
                --variant resnet50 \\
                --checkpoint working/RobustVideoMatting/rvm_resnet50.pth \\
                --input-source proc/{id}/sq_bw.mp4 \\
                --output-type png_sequence \\
                --output-alpha proc/{id}/mat_bw \\
                --seq-chunk 12
            """
    os.system(cmd)

    fw_frames = sorted(glob.glob(f"proc/{id}/mat_fw/*.png"))
    bw_frames = sorted(glob.glob(f"proc/{id}/mat_bw/*.png"))
    nframes = len(fw_frames)
    for i in range(nframes):
        fwf = cv2.imread(fw_frames[i], cv2.IMREAD_GRAYSCALE).astype(np.float32)
        bwf = cv2.imread(bw_frames[-i-1], cv2.IMREAD_GRAYSCALE).astype(np.float32)
        out = fwf * i/nframes + bwf * (nframes-i)/nframes
        out = 255 - out.astype(np.uint8)

        # out = cv2.threshold(out, 127, 255, cv2.THRESH_BINARY)[1]

        cv2.imwrite(f"proc/{id}/mat_mean/{i:04}.png", out)

# def crop_main_blob(id):
#     frames = sorted(glob.glob(f"proc/{id}/mat_mean/*.png"))
#     detector = cv2.SimpleBlobDetector()

#     for i in range(len(frames)):
        
#         # keypoints = detector.detect(im), cv2.IMREAD_GRAYSCALE



In [32]:
for ID in IDS[1:2]:
    crop_squares(ID)
    mat_squares(ID)
    # crop_main_blob(ID)

ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l